In [1]:
import pandas as pd
import multiprocessing as mp
import glob
from functools import partial, reduce
from functional import seq
import seaborn as sns

In [2]:
import numpy as np

# Load data

In [3]:
data_directory = "/bigdata/web_tracking/data/release/"

In [4]:
users = pd.read_csv(data_directory + '/raw/users.csv')

In [5]:
urls = pd.read_csv(
    data_directory + '/raw/browsing.csv', dtype={'id': int, 'web_visits_id': int}, parse_dates=['used_at'])

In [6]:
domain_categories = pd.read_csv(
    data_directory + '/raw/domain_categories.csv', converters={'category': lambda x: tuple(sorted(x.split(',')))})

# Defining the category of a website

Cleaning up a bit the data:

In [7]:
words_dashed = ['black', 'content', 'illegal', 'information', 'job', 'media', 
                'message', 'filter', 'real', 'search', 'social', 'streaming', 'virtual']

def put_dash(words_dashed, text):
    temp = list(filter(lambda w: text.startswith(w), words_dashed))
    return text if len(temp) < 1 else text.replace(temp[0], f'{temp[0]}-')


words_replaced = [('food', 'food and recipes'), ('filteravoidance', 'proxy and filter-avoidance'), 
                  ('translation', 'translators'), ('sport', 'sports')]

def replace(words_replaced, text):
    temp = list(filter(lambda p: p[0] in text, words_replaced))
    return text if len(temp) < 1 else temp[0][1]
    

dash = partial(put_dash, words_dashed)
words = partial(replace, words_replaced)

categories = pd.DataFrame(seq(domain_categories.category).reduce(lambda acc, i: acc.union( set(i)), set()), columns=['raw']) \
                .sort_values('raw') \
                .assign(name=lambda df: df.raw.apply(lambda x: words(dash(x.replace('and', ' and '))))) \
                .assign(id=lambda df: (df.name != df.name.shift()).cumsum())

#### Add Category Manually

We will add two new categories: email and productivity.

In [8]:
categories = categories.append(
    pd.DataFrame([(None, 'email', 0), (None, 'productivity', 0)], 
                 columns=['raw', 'name', 'id'])).sort_values(['name', 'raw']).assign(
    id=lambda df: (df.name != df.name.shift()).cumsum())

Let's export this:

In [9]:
categories.to_csv(data_directory + '/pre_processed/categories.csv', index=False)

#### Rebuild Domain Categories

- There was domain-category_ids match, now it's domain-category_names

In [10]:
def find_category_names(categories, t):
    return tuple(map(lambda c: categories.query(f"raw == '{c}'").iloc[0, 1], t))

category = partial(find_category_names, categories)
domain_categories = domain_categories.assign(category_names=domain_categories.category.apply(category))

Exporting the data:

In [11]:
domain_categories_processed = domain_categories[['domain', 'category_names']].assign(
    category_names=lambda df: df.category_names.apply(lambda l: ','.join(l)))

domain_categories_processed.to_csv(data_directory + '/pre_processed/processed_domain_categories.csv', index=False)

# Adding category info related to the subdomains

In [26]:
subdomains = urls[~urls.subdomain.isna()][["id", "url", "domain", "subdomain"]].copy()
subdomains = subdomains.merge(domain_categories_processed, on='domain')

### Examining 'mail' subdomains

1. Large companies which provide email services such as Google, Yahoo, T-Online, Vodafoen, GMX, Web.de.
2. Universities in Germany provide email services.
3. Other subdomains are seen in websites for the business purposes, i.e. subscription, login to buy, etc.

In [27]:
def get_subdomains(df, main, sub): 
    return df[(df.domain == main) & (df.subdomain.str.contains(sub))]

Let's see the ones that contains `mail`:

In [30]:
mail_subdomains = subdomains[subdomains.subdomain.str.contains('mail')]
top_mail_subdomains = mail_subdomains.groupby('domain', as_index=False).agg({'url': 'count'}).nlargest(400, 'url')
top_mail_subdomains.to_csv(data_directory + '/pre_processed/domains_containing_mail_raw.csv', index=False)

Next, we manually curated this .csv in order to generate a new file named `domains_contain_mail.csv`.

The method: 
- Look at the top domains with subdomain that has _mail_ string.
- If sample urls belong the domain seem meaningful urls, then check domain as email.
- Totally, 400 domain were evaluated manually.

### Creating subdomains of different types

* Emails Domains

In [33]:
urls_email = pd.concat(
    [
        subdomains[(subdomains.subdomain.str.endswith('mail.'))]
            .merge(pd.read_csv(data_directory + '/pre_processed/domains_containing_mail.csv', 
                               usecols=['domain', 'is_email']).query('is_email == 1'), on='domain')
            .drop(columns=['is_email']),
        get_subdomains(subdomains, 'google.com', 'accounts.'),
        get_subdomains(subdomains, 'web.de', 'navigator.'),
        get_subdomains(subdomains, 'yahoo.com', 'login.'),
        get_subdomains(subdomains, 'live.com', 'outlook.'),
        get_subdomains(subdomains, 'live.com', 'login.'),
        get_subdomains(subdomains, 'live.com', 'account.'),
        get_subdomains(subdomains, 'live.com', 'signup.'),
        get_subdomains(subdomains, 'gmx.net', 'navigator.'),
        get_subdomains(subdomains, 'aol.com', 'login.'),
    ],
    sort=False
) \
    .assign(
        sub_level_domain=lambda df: df.subdomain + df.domain,
        category_names=lambda df: df.category_names.apply(
            lambda x: tuple(['email']))
).drop_duplicates(['id'])

In [34]:
urls_email.shape

(970034, 6)

* Productivity domains

In [35]:
urls_productivity = pd.concat([
    get_subdomains(subdomains, 'google.com', 'docs.'),
    get_subdomains(subdomains, 'google.com', 'calendar.'),
    get_subdomains(subdomains, 'google.com', 'office.'),
],
    sort=False 
) \
    .assign(
        sub_level_domain=lambda df: df.subdomain + df.domain,
        category_names=lambda df: df.category_names.apply(
            lambda x: tuple(['productivity']))
).drop_duplicates(['id'])

urls_productivity.sample(5)

,id,url,domain,subdomain,category_names,sub_level_domain
163077,1039811007,230852,google.com,calendar.,"(productivity,)",calendar.google.com
464735,1075213104,2174152,google.com,docs.,"(productivity,)",docs.google.com
261579,1023653160,120291,google.com,calendar.,"(productivity,)",calendar.google.com
404420,1057529948,1823563,google.com,docs.,"(productivity,)",docs.google.com
139422,1034140416,18243,google.com,calendar.,"(productivity,)",calendar.google.com


In [36]:
urls_productivity.shape

(25244, 6)

* Media sharing domains

In [37]:
urls_media_sharing = pd.concat([
    get_subdomains(subdomains, 'google.com', 'photos.'),
    get_subdomains(subdomains, 'google.com', 'drive.'),
    get_subdomains(subdomains, 'live.com', 'onedrive.'),
],
    sort=False 
) \
    .assign(
        sub_level_domain=lambda df: df.subdomain + df.domain,
        category_names=lambda df: df.category_names.apply(
            lambda x: tuple(['media-sharing']))
).drop_duplicates(['id'])

urls_media_sharing.sample(5)

,id,url,domain,subdomain,category_names,sub_level_domain
217403,1013466324,92873,google.com,photos.,"(media-sharing,)",photos.google.com
296969,1028840884,1153843,google.com,photos.,"(media-sharing,)",photos.google.com
400526,1052449436,622005,google.com,photos.,"(media-sharing,)",photos.google.com
307356,1067644647,1236928,google.com,photos.,"(media-sharing,)",photos.google.com
142255,1035624776,55379,google.com,drive.,"(media-sharing,)",drive.google.com


In [38]:
urls_media_sharing.shape

(15375, 6)

* Social networking domains

In [39]:
urls_social_networking = pd.concat([
    get_subdomains(subdomains, 'google.com', 'plus.'),
    get_subdomains(subdomains, 'yahoo.com', 'drive.'),
    get_subdomains(subdomains, 'vodafone.de', 'groups.'),
],
    sort=False 
) \
    .assign(
        sub_level_domain=lambda df: df.subdomain + df.domain,
        category_names=lambda df: df.category_names.apply(
            lambda x: tuple(['social-networking']))
).drop_duplicates(['id'])

urls_social_networking.sample(5)

,id,url,domain,subdomain,category_names,sub_level_domain
552122,1084563179,294048,google.com,plus.,"(social-networking,)",plus.google.com
292631,1065274503,294048,google.com,plus.,"(social-networking,)",plus.google.com
290939,1028140244,1126675,google.com,plus.,"(social-networking,)",plus.google.com
263544,1060449007,294048,google.com,plus.,"(social-networking,)",plus.google.com
525748,1084953708,294048,google.com,plus.,"(social-networking,)",plus.google.com


In [40]:
urls_social_networking.shape

(2285, 6)

* News domains

In [41]:
urls_news = pd.concat([
    get_subdomains(subdomains, 'google.com', 'news.'),
    get_subdomains(subdomains, 'vodafone.de', 'magazin.'),
    get_subdomains(subdomains, 'vodafone.de', 'x.enews.'),
],
    sort=False 
) \
    .assign(
        sub_level_domain=lambda df: df.subdomain + df.domain,
        category_names=lambda df: df.category_names.apply(
            lambda x: tuple(['news']))
).drop_duplicates(['id'])

urls_news.sample(5)

,id,url,domain,subdomain,category_names,sub_level_domain
437478,1066834304,2008110,google.com,news.,"(news,)",news.google.com
224838,1053373951,412702,google.com,news.,"(news,)",news.google.com
364718,1044907736,25409,google.com,news.,"(news,)",news.google.com
230679,1054464871,681372,google.com,news.,"(news,)",news.google.com
153652,1036466839,25409,google.com,news.,"(news,)",news.google.com


In [42]:
urls_news.shape

(6165, 6)

* Translators domains

In [43]:
urls_translators = pd.concat([
    get_subdomains(subdomains, 'google.com', 'translate.'),
],
    sort=False 
) \
    .assign(
        sub_level_domain=lambda df: df.subdomain + df.domain,
        category_names=lambda df: df.category_names.apply(
            lambda x: tuple(['translators']))
).drop_duplicates(['id'])

urls_news.sample(5)

,id,url,domain,subdomain,category_names,sub_level_domain
205972,1050720711,144856,google.com,news.,"(news,)",news.google.com
361657,1044392684,25409,google.com,news.,"(news,)",news.google.com
478834,1070053127,145185,google.com,news.,"(news,)",news.google.com
143888,1036261080,25409,google.com,news.,"(news,)",news.google.com
427327,1063952688,25409,google.com,news.,"(news,)",news.google.com


In [44]:
urls_translators.shape

(28106, 6)

* Shopping domains

In [45]:
urls_shopping = pd.concat([
    get_subdomains(subdomains, 'google.com', 'play.'),
],
    sort=False 
) \
    .assign(
        sub_level_domain=lambda df: df.subdomain + df.domain,
        category_names=lambda df: df.category_names.apply(
            lambda x: tuple(['shopping']))
).drop_duplicates(['id'])

urls_shopping.sample(5)

,id,url,domain,subdomain,category_names,sub_level_domain
194801,1009590920,434260,google.com,play.,"(shopping,)",play.google.com
509325,1079769267,2470592,google.com,play.,"(shopping,)",play.google.com
151068,1035306895,18793,google.com,play.,"(shopping,)",play.google.com
317946,1032592752,94485,google.com,play.,"(shopping,)",play.google.com
533318,1087367540,2605791,google.com,play.,"(shopping,)",play.google.com


In [46]:
urls_shopping.shape

(3206, 6)

# Exporting data

Creating a new browsing history file:

In [62]:
urls_with_categories = urls.merge(domain_categories, on='domain').rename(columns={'domain': 'top_level_domain'})

In [63]:
urls_with_categories.head()

,web_visits_id,id,panelist_id,url,used_at,active_seconds,top_level_domain,subdomain,category,category_names
0,111761051,1033436752,1421,0,2018-10-05 22:02:38,4,ebesucher.de,NaN,"(business, education)","(business, education)"
1,111761051,1033436580,1421,0,2018-10-05 21:41:22,10,ebesucher.de,NaN,"(business, education)","(business, education)"
2,111761051,1033436588,1421,0,2018-10-05 21:45:36,5,ebesucher.de,NaN,"(business, education)","(business, education)"
3,111761051,1033436584,1421,0,2018-10-05 21:42:22,181,ebesucher.de,NaN,"(business, education)","(business, education)"
4,111761159,1033437168,1421,0,2018-10-05 23:48:14,6,ebesucher.de,NaN,"(business, education)","(business, education)"


In [64]:
urls_sublevel = pd.concat([
    urls_email,
    urls_productivity,
    urls_media_sharing,
    urls_social_networking,
    urls_news,
    urls_translators,
    urls_shopping
], sort=False).drop_duplicates(['id'])

urls_sublevel.shape

(1050415, 6)

In [65]:
urls_sublevel.head()

,id,url,domain,subdomain,category_names,sub_level_domain
0,1034500040,28061,live.com,bay174.mail.,"(email,)",bay174.mail.live.com
1,1037376788,97009,live.com,dub125.mail.,"(email,)",dub125.mail.live.com
2,1037377304,97009,live.com,dub125.mail.,"(email,)",dub125.mail.live.com
3,1035452643,134272,live.com,dub118.mail.,"(email,)",dub118.mail.live.com
4,1036819471,163690,live.com,dub129.mail.,"(email,)",dub129.mail.live.com


In [70]:
urls_with_subdomains = urls_with_categories.merge(
    urls_sublevel[['id', 'sub_level_domain', 'category_names']],
    on='id',
    how='left',
    suffixes=('_top', '_sub')
)

In [73]:
urls_with_subdomains = urls_with_subdomains.assign(
                domain = np.where(urls_with_subdomains.sub_level_domain.notnull(), 
                                  urls_with_subdomains.sub_level_domain, 
                                  urls_with_subdomains.top_level_domain),
                category_names = np.where(urls_with_subdomains.category_names_sub.notnull(), 
                                          urls_with_subdomains.category_names_sub, 
                                          urls_with_subdomains.category_names_top))

In [74]:
urls_with_subdomains.sample(5)

,web_visits_id,id,panelist_id,url,used_at,active_seconds,top_level_domain,subdomain,category,category_names_top,sub_level_domain,category_names_sub,domain,category_names
3355632,112461435,1036139439,750,148873,2018-10-06 20:32:39,6,ebay-kleinanzeigen.de,NaN,"(business, shopping)","(business, shopping)",NaN,NaN,ebay-kleinanzeigen.de,"(business, shopping)"
525092,118215504,1057628044,939,1825604,2018-10-15 19:51:49,24,ebay.de,NaN,"(business, shopping)","(business, shopping)",NaN,NaN,ebay.de,"(business, shopping)"
7322238,116272616,1031175976,1022,690076,2018-10-04 21:27:39,6,zattoo.com,NaN,"(business, entertainment)","(business, entertainment)",NaN,NaN,zattoo.com,"(business, entertainment)"
7553734,121144500,1073528195,2030,534759,2018-10-22 23:38:45,10,mailbang.de,NaN,"(business, entertainment)","(business, entertainment)",NaN,NaN,mailbang.de,"(business, entertainment)"
858499,114137492,1012199736,1095,544882,2018-10-01 11:21:49,16,google.de,NaN,"(searchenginesandportals,)","(search-engines and portals,)",NaN,NaN,google.de,"(search-engines and portals,)"


##### Constructing new Time Difference DataFrame of Consecutive Records per User

In [53]:
def construct_consecutive_url_dataframe(df):
    df = df.sort_values(['panelist_id', 'used_at'])
    df = df.assign(
            prev_id = df.id.shift().apply(lambda x: 0 if np.isnan(x) else int(x)) * (df.panelist_id == df.panelist_id.shift()),
            left_at = df.used_at + df.active_seconds.apply(lambda x: pd.Timedelta(seconds=x)),
        )
    df = df.assign(
            gap_seconds = (df.used_at - df.left_at.shift()).dt.total_seconds().apply(lambda x: 0 if np.isnan(x) else int(x)) * (df.panelist_id == df.panelist_id.shift())
                )[['id', 'prev_id', 'panelist_id', 'used_at', 'left_at', 'active_seconds', 'gap_seconds', 'top_level_domain', 'category_names_top',
        'sub_level_domain', 'subdomain', 'category_names_sub', 'domain',
        'category_names']]
    return df

In [75]:
urls_with_subdomains_gap = construct_consecutive_url_dataframe(urls_with_subdomains)

In [76]:
len(urls_with_subdomains_gap)

9151243

In [78]:
urls_with_subdomains_gap.sample(10)

,id,prev_id,panelist_id,used_at,left_at,active_seconds,gap_seconds,top_level_domain,category_names_top,sub_level_domain,subdomain,category_names_sub,domain,category_names
6393179,1081177039,1081177035,1810,2018-10-25 19:31:14,2018-10-25 19:31:24,10,4,floryday.com,"(shopping,)",NaN,NaN,NaN,floryday.com,"(shopping,)"
5136981,1014645548,1014645544,1631,2018-10-02 06:48:06,2018-10-02 06:48:16,10,0,kicker.de,"(news and media, sports)",NaN,tipp.,NaN,kicker.de,"(news and media, sports)"
7854148,1101908400,1101908396,1157,2018-10-31 16:04:19,2018-10-31 16:04:20,1,23,btn-muenzen.de,"(business, education, shopping)",NaN,nlt.,NaN,btn-muenzen.de,"(business, education, shopping)"
4871679,1075645195,1075645191,651,2018-10-24 13:02:25,2018-10-24 13:02:35,10,0,chefkoch.de,"(business, food and recipes)",NaN,NaN,NaN,chefkoch.de,"(business, food and recipes)"
2365318,1051212816,1051212812,1148,2018-10-12 05:35:16,2018-10-12 05:35:18,2,0,google.com,"(search-engines and portals,)",mail.google.com,mail.,"(email,)",mail.google.com,"(email,)"
2336115,1092455895,1092455891,2030,2018-10-30 07:03:48,2018-10-30 07:03:50,2,0,google.com,"(search-engines and portals,)",mail.google.com,mail.,"(email,)",mail.google.com,"(email,)"
4751615,1041026143,1041026139,786,2018-10-08 09:51:39,2018-10-08 09:51:45,6,0,comdirect.de,"(business, economy and finance)",NaN,kunde.,NaN,comdirect.de,"(business, economy and finance)"
8954368,1068241439,1068241435,1988,2018-10-19 21:30:30,2018-10-19 21:30:48,18,0,ratehase.de,"(games,)",NaN,NaN,NaN,ratehase.de,"(games,)"
1184990,1101027616,1101027612,1022,2018-10-31 10:45:03,2018-10-31 10:45:06,3,2,google.de,"(search-engines and portals,)",NaN,NaN,NaN,google.de,"(search-engines and portals,)"
7166571,1026141844,1026141840,1023,2018-10-03 08:04:15,2018-10-03 08:04:25,10,224,meinvz.net,"(business, education)",NaN,NaN,NaN,meinvz.net,"(business, education)"


In [79]:
urls_with_subdomains_gap.assign(
    category_names_top = lambda df: df.category_names_top.apply(lambda t: None if pd.isnull(t) else ','.join(t)),
    category_names_sub = lambda df: df.category_names_sub.apply(lambda t: None if pd.isnull(t) else ','.join(t)),
    category_names     = lambda df: df.category_names.apply(lambda t: None if pd.isnull(t) else ','.join(t)),
    ) \
    .to_csv(data_directory + "/pre_processed/browsing_with_gap.csv", index=False)